In [ ]:
from typing import List, Tuple
import itertools
import math
import time
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
from IPython.display import HTML
from tqdm import tqdm

In [ ]:
itertools.combinations?

In [ ]:


def iter_triangles(
    edge_length: int,
    start_offset: int = 1,
    exponent: int = 1,
):
    num_items = (edge_length - 1) * 3
    set_3 = set()
    for corners in itertools.combinations(range(num_items), 3):
        rest_numbers = set(i for i in range(num_items) if i not in corners)
        for edge1 in itertools.combinations(sorted(rest_numbers), (edge_length - 2)):
            for edge2 in itertools.combinations(sorted(rest_numbers - set(edge1)), (edge_length - 2)):
                for edge3 in itertools.combinations(sorted(rest_numbers - set(edge1) - set(edge2)), (edge_length - 2)):
                    #print(corners, edge1, edge2, edge3)
                    tri_3 = (corners[0], sum(edge1), corners[1], sum(edge2), corners[2], sum(edge3))
                    if tri_3 in set_3:
                        continue
                    set_3.add(tri_3)
                    set_3.add((*tri_3[edge_length-1:], *tri_3[:edge_length-1]))
                    set_3.add((*tri_3[-edge_length+1:], *tri_3[:-edge_length+1]))
                    
                    yield tuple(
                        (v + start_offset) ** exponent
                        for v in (corners[0], *edge1, corners[1], *edge2, corners[2], *edge3)          
                    )
                
len(set(tqdm(iter_triangles(5))))

In [ ]:
len(list(itertools.combinations(range(4), 4)))

In [ ]:
math.factorial(6), 6**3

In [ ]:

def draw_triangle(image: PIL.Image.Image, indices: Tuple[int, ...], offset=(0, 0), scale=50):
    edge_length = len(indices)//3 + 1
    draw = PIL.ImageDraw.ImageDraw(image)
    font = PIL.ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeSans.ttf", scale / 4)
    for i, e in enumerate(indices):
        if i < edge_length:
            tx, ty = i, i
        elif i < edge_length * 2 - 1:
            tx, ty = i, edge_length * 2 - i - 2
        else:
            tx, ty = (edge_length - (i-edge_length*2)*2 - max(0, 6-edge_length) - max(0, len(str(e))-1.5)), 0
        draw.text((
            offset[0] + tx * scale / edge_length, 
            offset[1] + scale - scale/4+3 - ty * scale / edge_length
        ), str(e), fill=255, font=font)

image = PIL.Image.new("L", (100, 50), 0)
draw_triangle(image, list(range(1, 16)))
image

In [ ]:
def find_magic_triangles(
        edge_length: int = 3,
        start_offset: int = 1,
        scale: int = 50,
        exponent: int = 1,
        verbose: bool = True,
):
    num_items = (edge_length - 1) * 3
    magics = {}
    num_triangles = 0
    try:
        for indices in tqdm(
                iter_triangles(edge_length=edge_length, start_offset=start_offset, exponent=exponent), 
                disable=not verbose,
        ):
            num_triangles += 1
            edge1 = indices[:edge_length]
            edge2 = indices[edge_length - 1: edge_length * 2 - 1]
            sum1 = sum(edge1)
            sum2 = sum(edge2)
            if sum2 != sum1:
                continue
            edge3 = indices[edge_length * 2 - 2:] + indices[:1]
            sum3 = sum(edge3)
            if sum3 != sum1:
                continue
            magics.setdefault(sum1, []).append(indices)
    except KeyboardInterrupt:
        print("INTERRUPTED!")
        pass
    #magics = {0: [(1, 2, 3, 4, 5, 6)]}
    if verbose:
        time.sleep(.5)
    display(HTML(
        f"""<h3>Seitenlänge: {edge_length}, Zahlen: {start_offset} - {num_items - 1 + start_offset},
        Magische Dreiecke: {sum(len(t) for t in magics.values())}</h3><h4>aus {num_triangles:,} ungespiegelten/unrotierten
        von {math.factorial(num_items):,} möglichen Dreiecken</h4>"""
    ))
    for key in sorted(magics):
        edges = magics[key]
        corner_sums = set()
        for edge in edges:
            corner_sums.add(edge[0] + edge[edge_length-1] + edge[-edge_length+1])
        display(HTML(f"Kantensumme: <b>{key}</b>, Magische Dreiecke: <b>{len(edges)}</b>, Eckensumme: <b>{', '.join(str(s) for s in sorted(corner_sums))}</b>"))

        num_x = min(10, len(edges))
        num_y = (len(edges) + num_x - 1) // num_x
        image = PIL.Image.new("L", (scale * num_x * 2, int(scale * num_y * 1.2)), 0)
        for j, edge in enumerate(edges):
            ox = (j % num_x) * scale * 2
            oy = (j // num_x) * scale * 1.2
            draw_triangle(image, edge, offset=(ox, oy), scale=scale)
            #draw.text((ox * scale * 2 + tx * scale / edge_length, scale - 12 - ty * scale / edge_length), str(e), fill=255)
                    
        display(image)
find_magic_triangles(4, exponent=1, start_offset=1)

In [ ]:
find_magic_triangles(3, exponent=1, start_offset=1, verbose=False)
find_magic_triangles(4, exponent=1, start_offset=1, verbose=False)
find_magic_triangles(5, exponent=1, start_offset=1, verbose=False)

In [ ]:
find_magic_triangles(6, exponent=1, start_offset=1, verbose=False)

In [ ]:
for i in [3, 4, 5, 6]:
    find_magic_triangles(i, exponent=3, start_offset=1, verbose=False)